In [1]:
import os
import json

from typing import List, Dict, Any, Tuple, Optional

from formating import convert_json_to_insert_sql, convert_json_to_update_sql
from crud import insert_records, delete_records, fetch_records, get_tables

In [2]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
assets_dir = os.path.join(parent_dir, '_assets')
db_dir = os.path.join(parent_dir, 'fetusapp')
db_filename = 'data.sqlite'
data_filename = 'detoraki_fetusdata.json'
mapping_filename = 'column_mapping.json'

db_path = os.path.join(db_dir, db_filename)
json_file = os.path.join(assets_dir, data_filename)
mapping_file = os.path.join(current_dir, mapping_filename)

Load the source data

In [3]:
with open(json_file, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

Load the column mapping from the JSON file

In [4]:
with open(mapping_file, 'r', encoding='utf-8') as f:
        column_mapping = json.load(f)

### Setup admin in Users (INSERT)

The script should already be there in _assets

In [ ]:
target_tablename = 'users'
insert_records(target_tablename, db_path)


In [ ]:
fetch_records(target_tablename, db_path)

In [ ]:
# delete_records(target_tablename, db_path)

---------------------

### Migrate Patients (INSERT)

Includes manual step to treat patients missing last_name

In [ ]:
source_tablename = 's_org_ext'
target_tablename = 'patients'
data_index = 3

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

#### Data Cleansing:
* Identify records with null last name and either 
    * convert the null to the string 'null' in the generated script or
    * remove the record altogether if it doesn't have any useful payload

In [ ]:
patient_data = json_data[data_index]['data']
null_lastname_ids = [item['s_org_extID'] for item in patient_data if item['lastName'] == ""]
print(null_lastname_ids)

In [ ]:
delete_records(target_tablename, db_path)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

---------------------

### Migrate MedicalHistory (INSERT)

In [ ]:
source_tablename = 'tblmedhistory'
target_tablename = 'history_medical'
data_index = 10

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
# delete_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

------------------

### Migrate Short Gynaecological History data (UPDATE)

In [ ]:
source_tablename = 'tblgynhistoryshort'
target_tablename = 'history_medical'
data_index = 7

sql_output = convert_json_to_update_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename, source_id = 'MedHistoryID', target_id = 'id')
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)

---------

### Migrate Pathologies (UPDATE)

In [ ]:
source_tablename = 'tblpathologies'
target_tablename = 'history_medical'
data_index = 13

sql_output = convert_json_to_update_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename, source_id = 'MedHistoryID', target_id = 'id')
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

----------

### Migrate ObstetricsHistory (INSERT)

In [5]:
source_tablename = 'tblobstetrichistory'
target_tablename = 'history_obstetrics'
data_index = 11

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
delete_records(target_tablename, db_path)

In [6]:
insert_records(target_tablename, db_path)

Executing statement: INSERT INTO history_obstetrics (id, patient_id, ft, kt, embrioulkia, te, ae, created_on, created_by, last_updated_on, last_updated_by, is_active)
VALUES
(1, 3, 2, 0, 0, 0, 0, null, 1, '2013-11-21 18:05:33', 1, 1),
(5, 66, 2, 0, 0, 3, 0, '2014-02-27 01:50:24', 1, '2014-02-27 01:50:39', 1, 1),
(6, 85, 3, 0, 0, 0, 0, '2014-05-05 14:51:40', 1, '2014-05-05 14:51:40', 1, 1),
(7, 90, 0, 0, 0, 0, 0, '2014-06-16 14:24:00', 1, '2014-06-16 14:24:00', 1, 1),
(8, 92, 2, 0, 0, 0, 1, '2014-06-16 16:56:40', 1, '2014-06-16 16:56:40', 1, 1),
(9, 93, 2, 0, 0, 1, 1, '2014-06-19 10:06:02', 1, '2014-06-19 10:06:02', 1, 1),
(10, 97, 0, 0, 0, 1, 0, '2014-07-22 17:14:06', 1, '2014-07-22 17:14:06', 1, 1),
(11, 98, 0, 0, 0, 1, 0, '2014-07-24 13:37:30', 1, '2014-07-24 13:37:30', 1, 1),
(12, 103, 2, 0, 0, 1, 0, '2014-09-16 19:14:21', 1, '2014-09-16 19:14:21', 1, 1),
(13, 110, 2, 0, 1, 0, 0, '2014-10-14 13:03:07', 1, '2014-10-14 13:03:07', 1, 1),
(14, 112, 0, 1, 0, 2, 0, '2014-10-16 16:24:23', 

In [7]:
fetch_records(target_tablename, db_path)

(1, 3, 1, 1, None, '2013-11-21 18:05:33', 1, 2, 0, 0, 0, 0)
(5, 66, 1, 1, '2014-02-27 01:50:24', '2014-02-27 01:50:39', 1, 2, 0, 0, 3, 0)
(6, 85, 1, 1, '2014-05-05 14:51:40', '2014-05-05 14:51:40', 1, 3, 0, 0, 0, 0)
(7, 90, 1, 1, '2014-06-16 14:24:00', '2014-06-16 14:24:00', 1, 0, 0, 0, 0, 0)
(8, 92, 1, 1, '2014-06-16 16:56:40', '2014-06-16 16:56:40', 1, 2, 0, 0, 0, 1)


... to be continued

In [ ]:
get_tables(db_path)

--------------